样本分布不均衡就是指样本差异非常大，样本类别不均衡从数据量上可以分为大数据分布不均衡和小数据分布不均衡
+ 大数据分布不均：整体数据规模大，其中小样本类占比较少。但是从每个特征的分布来看，小样本也覆盖了大部分或全部的特征。
+ 小数据分布不均：这种情况数据整体规模小，并且占据少量样本比例的分类数据也少，这会导致特征分布的严重不平衡。例如共1000条数据样本的数据集中，其中占有10条样本分类，其特征无论如何你和也无法实现完整特征值的覆盖，此时属于严重的样本分布不均衡。

样本不均衡将导致样本量少的分类所包含的特征过少，很难从中提取规律，即使得到分类模型，也容易产生过度依赖于有限的数量样本而导致过拟合问题，当模型应用到新的数据上时，模型的准确性和健壮性将会很差。
样本不均衡是指不同样本之间比例的差异，差异达到10倍就需要引起警惕，如果超过20倍就一定要解决。

## 出现样本不均衡场景
+ 异常检测 大多数企业中的异常个案都是少量的，比如恶意刷单、黄牛订单、信用卡欺诈、电力窃电、设备故障等，这些数据样本占比通常是整体的很小一部分
+ 客户流失 大型企业的流失客户相对于整体客户而言通常是少量的，尤其是具有垄断地位的行业巨擘
+ 罕见事件 罕见事件与异常检测类似，但是罕见事件无法预测，随机发生
+ 发生频率低的事件 不同于罕见事件这类事件是有预期的计划之内，只不过很少发生，如双十一的营业额暴增。

## 解决数据不均衡
+ 过抽样 通过增加分类中样本较少的类别的采样数量来实现平衡，最直接的方法是简单复制小样本数据，缺点是如果特征少，会导致过拟合的问题。经过改进的过抽样方法通过在少数类中加入随机噪声、干扰数据或通过一定规则产生新的合成样本。
+ 欠抽样 通过减少分类中多数类样本的数量来实现样本均衡，最直接的方法是随机去掉一些多数类样本来减小多数类的规模，缺点是会丢失多数类中的一些重要信息。
+ 权重 对不同样本数量的类别赋予不同的权重（通常会设置为与样本量成反比） 
+ 集成方法 每次生成训练集时使用所有分类中的小样本量，同时从分类中的大样本量中随机抽取数据来与小样本量合并构成训练集，这样反复多次会得到很多训练集和训练模型。最后在应用时，使用组合方法（例如投票、加权投票等）产生分类预测结果。这种方法类似于随机森林。缺点是，比较吃计算资源，费时。

In [16]:
# 生成不平衡分类数据集
from collections import Counter
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=3000, n_features=2, n_informative=2,
                           n_redundant=0, n_repeated=0, n_classes=3,
                           n_clusters_per_class=1,
                           weights=[0.1, 0.05, 0.85],
                           class_sep=0.8, random_state=2018)
Counter(y)

Counter({2: 2532, 1: 163, 0: 305})

In [17]:
# 使用RandomOverSampler从少数类的样本中进行随机采样来增加新的样本使各个分类均衡
from imblearn.over_sampling import RandomOverSampler
 
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_sample(X, y)
sorted(Counter(y_resampled).items())

[(0, 2532), (1, 2532), (2, 2532)]

In [18]:
# SMOTE: 对于少数类样本a, 随机选择一个最近邻的样本b, 然后从a与b的连线上随机选取一个点c作为新的少数类样本
from imblearn.over_sampling import SMOTE
 
X_resampled_smote, y_resampled_smote = SMOTE().fit_sample(X, y)
 
sorted(Counter(y_resampled_smote).items())

[(0, 2532), (1, 2532), (2, 2532)]

In [19]:
# ADASYN: 关注的是在那些基于K最近邻分类器被错误分类的原始样本附近生成新的少数类样本
from imblearn.over_sampling import ADASYN

X_resampled_adasyn, y_resampled_adasyn = ADASYN().fit_sample(X, y)
 
sorted(Counter(y_resampled_adasyn).items())

[(0, 2522), (1, 2520), (2, 2532)]

In [21]:
# RandomUnderSampler函数是一种快速并十分简单的方式来平衡各个类别的数据: 随机选取数据的子集.
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0)
X_resampled, y_resampled = rus.fit_sample(X, y)
 
sorted(Counter(y_resampled).items())

[(0, 163), (1, 163), (2, 163)]

In [23]:
# 在之前的SMOTE方法中, 当由边界的样本与其他样本进行过采样差值时, 很容易生成一些噪音数据. 因此, 在过采样之后需要对样本进行清洗. 
# 这样TomekLink 与 EditedNearestNeighbours方法就能实现上述的要求.
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_sample(X, y)
 
sorted(Counter(y_resampled).items())

[(0, 2111), (1, 2099), (2, 1893)]

In [24]:
from imblearn.combine import SMOTETomek
smote_tomek = SMOTETomek(random_state=0)
X_resampled, y_resampled = smote_tomek.fit_sample(X, y)
 
sorted(Counter(y_resampled).items())

[(0, 2412), (1, 2414), (2, 2396)]

In [25]:
# 使用SVM的权重调节处理不均衡样本
from sklearn.svm import SVC  
svm_model = SVC(class_weight='balanced')
svm_model.fit(X, y)

D:\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [26]:
# EasyEnsemble 通过对原始的数据集进行随机下采样实现对数据集进行集成.
# EasyEnsemble 有两个很重要的参数: (i) n_subsets 控制的是子集的个数 and (ii) replacement 决定是有放回还是无放回的随机采样.
from imblearn.ensemble import EasyEnsemble
ee = EasyEnsemble(random_state=0, n_subsets=10)
X_resampled, y_resampled = ee.fit_sample(X, y)
sorted(Counter(y_resampled[0]).items())

[(0, 163), (1, 163), (2, 163)]

In [27]:
# BalanceCascade(级联平衡)的方法通过使用分类器(estimator参数)来确保那些被错分类的样本在下一次进行子集选取的时候也能被采样到. 同样, n_max_subset 参数控制子集的个数, 以及可以通过设置bootstrap=True来使用bootstraping(自助法).
from imblearn.ensemble import BalanceCascade
from sklearn.linear_model import LogisticRegression
bc = BalanceCascade(random_state=0,
                    estimator=LogisticRegression(random_state=0),
                    n_max_subset=4)
X_resampled, y_resampled = bc.fit_sample(X, y)
 
sorted(Counter(y_resampled[0]).items())


D:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
D:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
D:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to

[(0, 163), (1, 163), (2, 163)]

In [29]:
# BalancedBaggingClassifier 允许在训练每个基学习器之前对每个子集进行重抽样. 简而言之, 该方法结合了EasyEnsemble采样器与分类器(如BaggingClassifier)的结果.
from sklearn.tree import DecisionTreeClassifier
from imblearn.ensemble import BalancedBaggingClassifier
bbc = BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(),
                                ratio='auto',
                                replacement=False,
                                random_state=0)
bbc.fit(X, y) 

BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
             bootstrap=True, bootstrap_features=False, max_features=1.0,
             max_samples=1.0, n_estimators=10, n_jobs=1, oob_score=False,
             random_state=0, ratio='auto', replacement=False,
             sampling_strategy='auto', verbose=0, warm_start=False)